### Functionality 1 - Find the Neighbours!
It takes in input:

a node v

One of the following distances function: t(x,y), d(x,y) or network distance (i.e. consider all edges to have weight equal to 1).

a distance threshold d

Implement an algorithm (using proper data structures) that returns the set of nodes at distance <= d from v, corresponding to v’s neighborhood.

In [42]:
import pandas as pd

In [43]:
node_info= pd.read_csv("node_information_file.csv")
travel_time= pd.read_csv("travel_time_graph.csv")
spatial_distance= pd.read_csv("distance_graph.csv")

In [44]:
node_info.columns=["id_node", "lat", "long"]

In [45]:
spatial_distance.columns=['Node1', 'Node2', 'distance']
travel_time.columns=['Node1', 'Node2', 'distance']

from pandas import DataFrame
network_df=DataFrame(columns= ['Node1', 'Node2','network_distance'])
network_df['Node1'] = pd.Series(travel_time['Node1'])
network_df['Node2'] = pd.Series(travel_time['Node2'])
network_df['network_distance']=1

In [46]:
network_df.columns=['Node1', 'Node2', 'distance']

In [47]:
node_info['lat'] = node_info['lat']/(1000000)
node_info['long'] = node_info['long']/(1000000)

In [48]:
node_info_p=node_info[:500]
node_info_p.head(2)

,id_node,lat,long
0,1,-114.315309,34.133550
1,2,-114.223946,34.176221


In [61]:
from tqdm import tqdm_notebook

v= int(input("Choose a starting node: "))
distance=input("Choose a type of distance: ")
max_depth=int(input("Insert a treshold value: "))   
    
def search_neighbour(v, distance, max_depth): 
    
    if distance == "spatial":
        df = spatial_distance
    elif distance == "time":
        df = travel_time
    elif distance == "network":
        df = network_df

    neighbour = df[(df['Node1'] == v) & (df['distance'] <= max_depth)]
    l_n = len(neighbour)

    if l_n>0:
        listaNodi2 = neighbour['Node2'].tolist()
        for nodo2 in listaNodi2:
            d = df[(df['Node1'] == v) & (df['Node2'] == nodo2)]
            d=int(d['distance'])
            neighbour=pd.concat([neighbour,search_neighbour(nodo2, distance, max_depth-d)]).drop_duplicates().reset_index(drop=True)
    return neighbour



neighbour=search_neighbour(v,distance,max_depth)
result = set(neighbour['Node2'].tolist())

print(result)

Choose a starting node: 2
Choose a type of distance: spatial
Insert a treshold value: 3000
{1050176, 2, 1048578, 1050180, 1990, 1991, 1050174, 1993, 1994, 1995, 1050774, 1050172, 2462, 1050175}


### Visualization 1 - Find the Neighbours!
Once the user runs Functionality 1, we want the system to show in output a complete map that contains: the input node, the output nodes and all the streets that connect these points. Choose different colors in order to highlight which is the input node, and which are the output nodes. Furthermore, choose different colors for edges, according to the distance function used.

In [62]:
import json
import urllib
import pandas as pd 
import numpy as np
from io import StringIO 
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import collections
import heapq
import geopy
from geopy.geocoders import Nominatim, GoogleV3
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json, string
import requests
import geocoder
from folium import Map, Marker, GeoJson, LayerControl
from ediblepickle import checkpoint
from tqdm import tqdm_notebook
import os 
import folium
from folium import plugins
%matplotlib inline

In [64]:
node_info['Node1']=node_info['id_node']
neighbour=pd.merge(node_info, neighbour, on='Node1', how='inner')

In [65]:
node_info['Node2']=node_info['id_node']

In [66]:
neighbour.rename(columns={'Node2_y':'Node2'}, inplace=True)

In [67]:
neighbour=pd.merge(node_info,neighbour, on='Node2', how='inner')

In [81]:
neighbour['lat_x'][0]

-114.223946

In [68]:
neighbour

,id_node_x,lat_x,long_x,Node1_x,Node2,id_node_y,lat_y,long_y,Node1_y,Node2_x,distance
0,2,-114.223946,34.176221,2,2,2462,-114.223670,34.176668,2462,2462,558
1,2,-114.223946,34.176221,2,2,1050174,-114.223863,34.175877,1050174,1050174,390
2,1990,-114.224964,34.175694,1990,1990,1993,-114.224496,34.175740,1993,1993,433
3,1990,-114.224964,34.175694,1990,1990,1050172,-114.225570,34.175602,1050172,1050172,566
4,1991,-114.226177,34.177662,1991,1991,1048578,-114.225874,34.177662,1048578,1048578,279
5,1993,-114.224496,34.175740,1993,1993,1990,-114.224964,34.175694,1990,1990,433
6,1993,-114.224496,34.175740,1993,1993,1050174,-114.223863,34.175877,1050174,1050174,602
7,1994,-114.224691,34.174756,1994,1994,1990,-114.224964,34.175694,1990,1990,1072
8,1995,-114.223180,34.174207,1995,1995,1050175,-114.223009,34.175191,1050175,1050175,1105
9,2462,-114.223670,34.176668,2462,2462,2,-114.223946,34.176221,2,2,558


In [97]:
map = folium.Map(location=[np.median((neighbour['long_x']).tolist()),
                           np.median((neighbour['lat_x']).tolist())], 
                 default_zoom_start=120)#keep the median to localize our data

for i in tqdm_notebook(range(len(neighbour))):
    
    folium.features.RegularPolygonMarker(location = [(neighbour['long_x'].values)[0], 
                                                     (neighbour['lat_x'].values)[0]],
                                             number_of_sides = 3,
                                             popup='Starting node',
                                             fill_color='red',
                                             radius = 15,
                                             weight = 4,
                                             fill_opacity = 0.8).add_to(map)

    

    


    folium.features.RegularPolygonMarker(location = [(neighbour['long_x'].values)[i], 
                                                         (neighbour['lat_x'].values)[i]],
                                                 number_of_sides = 3,
                                                 radius = 10,
                                                 weight = 4,
                                                 fill_opacity = 0.8).add_to(map)

#to add the edge between two nodes
for i in tqdm_notebook(range(len(neighbour))):
    folium.PolyLine(locations = [(((neighbour['long_x']).values)[i], 
                                  ((neighbour['lat_x']).values)[i]), 
                                 (((neighbour['long_y']).values)[i], 
                                  ((neighbour['lat_y']).values)[i])], 
                    line_opacity = 0.5).add_to(map) 
#weight represent the dimension of the node
map.save("mapneighbours.html")
 
map